In [1]:
import datetime

In [2]:
def getNextTargetDate(dataframe, current_date, days_ahead=7):
    next_week = dataframe.loc[current_date:current_date + datetime.timedelta(days=days_ahead)]
    next_datetime_str = str(next_week.head(2).tail(1).index.values[0])
    next_date_str = next_datetime_str.split('T')[0]
    next_date = datetime.datetime.strptime(next_date_str, '%Y-%m-%d')
    return next_date

In [4]:
def str_to_datetime(date_str):
    year, month, day = map(int, date_str.split('-'))
    return datetime.datetime(year=year, month=month, day=day)

print(str_to_datetime('2025-03-21'))

2025-03-21 00:00:00


In [ ]:
def window_df(first_date, last_date,n=3):
    first_date = str_to_datetime(first_date)
    last_date = str_to_datetime(last_date)

    target_date = first_date

    last_time = False
    while True:
        next_date = getNextTargetDate

    return first_date > last_date

In [10]:
print(window_df('2023-02-03', '2025-03-22'))

False
